In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [44]:
df = pd.read_pickle('../data/interim/1.0-ji-initial-data-cleaned.pkl')

In [45]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 36081 entries, 1 to 54998
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Transaction ID       32466 non-null  object        
 1   Customer ID          32503 non-null  object        
 2   Transaction Date     32473 non-null  datetime64[ns]
 3   Operator Name        36081 non-null  category      
 4   Transaction Type     36081 non-null  category      
 5   Transaction Amount   32507 non-null  float64       
 6   Customer Age         32625 non-null  float64       
 7   Customer Gender      32501 non-null  category      
 8   Customer Location    32537 non-null  object        
 9   Service Plan         36081 non-null  category      
 10  Data Usage (MB)      32427 non-null  float64       
 11  Call Duration (min)  32548 non-null  float64       
 12  SMS Sent             32441 non-null  float64       
 13  Internet Package     36081 non-null 

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
1,21ca8795-8ad9-47bc-a16b-e4654697ee52,NaN,2023-10-29 02:48:20,9mobile,Bill Payment,4804.10,39.0,other,Kaduna,Postpaid,3440.00,234.30,13.0,Weekly,Failed
2,ea6478bf-6a8c-4d91-bd63-b4464ebc5ae0,3052574b-ae63-4e67-8d86-5f8418ef6f66,2024-04-03 02:22:20,9mobile,Bill Payment,19303.03,NaN,male,Kaduna,Prepaid,NaN,0.90,0.0,Weekly,Completed
3,NaN,15ad55e7-2c9e-4a84-9b9d-9e7d7747a177,2024-02-07 00:22:30,9mobile,Bill Payment,4094.00,NaN,male,Owerri,Prepaid,2250.00,16.38,20.0,Daily,Completed
4,a1165572-b0f4-42e2-9890-eb16ef0b8741,a4adc3b5-2b36-4ba5-b7d9-1a103bfb247a,2008-10-31 21:12:40,Glo,Data Purchase,NaN,36.0,female,Sokoto,Prepaid,3307.97,92.61,58.0,Weekly,Failed
7,07c57156-2d68-433b-92c2-8839db514bc0,NaN,NaT,Glo,Data Purchase,4499.60,49.0,male,Ilorin,Prepaid,3706.69,3.42,59.0,Daily,Pending


**Checking for duplicates in the dataset**

In [46]:
df.duplicated().sum()

np.int64(3272)

In [47]:
# Quick view of duplicated data

df[df.duplicated(keep=False)].sort_values(by='Transaction Date').head()

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
38004,4c31d765-c7e4-4b50-b3ff-11320291d8dc,07220b76-0cf5-4fab-b850-b81835fd8a37,1970-01-08 18:47:21,Airtel,Data Purchase,6877.00,64.0,female,Makurdi,Prepaid,1580.41,197.24,96.0,Monthly,Failed
15485,4c31d765-c7e4-4b50-b3ff-11320291d8dc,07220b76-0cf5-4fab-b850-b81835fd8a37,1970-01-08 18:47:21,Airtel,Data Purchase,6877.00,64.0,female,Makurdi,Prepaid,1580.41,197.24,96.0,Monthly,Failed
31039,3a934b73-e4bc-476a-8e22-36cf06972dbc,47bd1cee-56ad-4a0e-915b-0c52bfa08d88,1970-01-21 12:02:36,9mobile,Data Purchase,NaN,28.0,female,Awka,Prepaid,1169.26,222.92,31.0,Monthly,Failed
52044,3a934b73-e4bc-476a-8e22-36cf06972dbc,47bd1cee-56ad-4a0e-915b-0c52bfa08d88,1970-01-21 12:02:36,9mobile,Data Purchase,NaN,28.0,female,Awka,Prepaid,1169.26,222.92,31.0,Monthly,Failed
40977,399910a7-4d07-4910-8e1c-01a336b7c131,6895bb13-8211-4faf-9e99-c98472754f58,1970-03-15 10:36:24,9mobile,Data Purchase,16996.59,19.0,female,Aba,Postpaid,4242.63,277.34,47.0,Monthly,Failed


In [48]:
# Dropping duplicates
df = df.drop_duplicates()

In [49]:
df.shape

(32809, 15)

**Dropping unncessary columns**

The Transaction ID and Customer ID Columns do not appear to be valuable for future analytics so we can drop them

In [50]:
df[['Transaction ID','Customer ID']].nunique()

Transaction ID    29524
Customer ID       29536
dtype: int64

In [51]:
df = df.drop(columns=['Transaction ID','Customer ID'])

**Dealing with missing values**

In [52]:
df.isna().sum()

Transaction Date       3321
Operator Name             0
Transaction Type          0
Transaction Amount     3249
Customer Age           3134
Customer Gender        3259
Customer Location      3247
Service Plan              0
Data Usage (MB)        3310
Call Duration (min)    3231
SMS Sent               3305
Internet Package          0
Transaction Status        0
dtype: int64

The question here wether to perform some imputation method for the missing values or just to drop them. I would rather much prefer to drop them since we will still have a substantial dataset after dropping them because performing some data imputation method means we are creating our own data which is not commensurate with the real data.

In [53]:
df = df.dropna()

In [54]:
df.shape

(14133, 13)

**Refactoring the columns to lowercase and using underscore**

In [66]:
df.columns = (
                df.columns
                .str.replace(' ','_')
                .str.lower()
                )

In [67]:
df.head()

,transaction_date,operator_name,transaction_type,transaction_amount,customer_age,customer_gender,customer_location,service_plan,data_usage_(mb),call_duration_(min),sms_sent,internet_package,transaction_status
1,2023-10-29 02:48:20,9mobile,Bill Payment,4804.10,39.0,other,Kaduna,Postpaid,3440.00,234.30,13.0,Weekly,Failed
8,2024-03-02 16:58:08,Airtel,Bill Payment,4190.81,20.0,female,Kano,Postpaid,4624.89,250.80,75.0,Weekly,Completed
10,2023-10-22 19:25:17,Glo,Airtime Purchase,13647.70,32.0,male,Gombe,Prepaid,3653.14,273.39,47.0,Daily,Pending
15,2024-07-05 02:27:15,9mobile,Airtime Purchase,13246.30,30.0,male,Minna,Postpaid,2340.04,4.00,56.0,Daily,Completed
19,1971-01-09 15:49:16,Glo,Bill Payment,12112.00,38.0,male,Asaba,Postpaid,1501.59,33.21,74.0,Daily,Completed


**Getting an overview of the statistical spread of the dataset**

In [55]:
# Number type statistical overview
df.describe()

,Transaction Date,Transaction Amount,Customer Age,Data Usage (MB),Call Duration (min),SMS Sent
count,14133,14133.000000,14133.000000,14133.000000,14133.000000,14133.000000
mean,2018-09-12 09:52:53.963277568,9998.714838,44.039624,2491.107208,149.450257,49.406495
min,1970-01-08 18:47:21,101.030000,18.000000,0.000000,0.000000,0.000000
25%,2023-08-25 17:00:53,5014.550000,31.000000,1222.660000,75.000000,24.000000
50%,2023-12-17 23:46:31,9997.500000,44.000000,2478.160000,148.860000,49.000000
75%,2024-04-09 21:19:02,14927.730000,57.000000,3754.000000,224.770000,74.000000
max,2024-07-31 12:46:55,19998.600000,70.000000,5000.000000,300.000000,100.000000
std,NaN,5735.762858,15.277015,1450.120789,86.418473,29.049537


In [57]:
# Categorical type statistical overview
df.select_dtypes('category').describe()

,Operator Name,Transaction Type,Customer Gender,Service Plan,Internet Package,Transaction Status
count,14133,14133,14133,14133,14133,14133
unique,4,3,3,2,3,3
top,MTN,Airtime Purchase,female,Prepaid,Monthly,Completed
freq,3569,4759,5674,7100,4759,4738


Everything seems to be cleaned within reason

In [68]:
# Saving file to csv
df.to_csv('../data/processed/2.0-ji-final-cleaned-data.csv', index=False)

# Saving file to pickle
df.to_pickle('../data/processed/2.0-ji-final-cleaned-data.pkl')
